<a href="https://colab.research.google.com/github/liuchiente/data_science/blob/main/%E4%BB%A5Pearson%E5%87%BD%E6%95%B8%E5%AF%A6%E7%8F%BE%E9%9B%BB%E5%BD%B1%E6%8E%A8%E8%96%A6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 準備環境

1.   掛載硬碟
2.   安裝套件
3.   引入函式庫

In [2]:
#掛載硬碟
SYS_DIR = "/content/drive/My Drive/Colab Notebooks/MovieLens"
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [7]:
#引入函式庫
import numpy as np
import pandas as pd
import torch
from torch import nn
from torchtext import data, datasets
import nltk
import os
import zipfile

In [4]:
#切換預設工作目錄切換預設工作目錄
if not os.path.isdir(SYS_DIR):
    os.mkdir(SYS_DIR)
os.chdir(SYS_DIR)

# 資料處理

1.  下載資料
2.  資料清洗

In [ ]:
#下載資料集
!wget http://files.grouplens.org/datasets/movielens/ml-1m.zip

In [5]:
with zipfile.ZipFile('ml-1m.zip', 'r') as zip_ref:
    zip_ref.extractall('ml-1m')

In [ ]:
#資料處理
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table('ml-1m/ml-1m/users.dat', sep='::', header=None, names=unames, engine='python')
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table('ml-1m/ml-1m/ratings.dat', sep='::', header=None, names=rnames, engine='python')
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('ml-1m/ml-1m/movies.dat', sep='::', header=None, names=mnames, engine='python')


In [ ]:
#載入預測資料
snames = ['movie_id', 'title', 'genres','rating']
sample = pd.read_table('sample.dat', sep='::', header=None, names=snames, engine='python')
tnames = ['movie_id', 'title', 'genres','rating']
test = pd.read_table('test.dat', sep='::', header=None, names=tnames, engine='python')

In [ ]:
#看預測資料集
#合併資料集原本評分、預測資料集
pred_user=pd.concat([sample,test],axis=0)
pred_movie_list=list(pred_user['movie_id'])

In [ ]:
#找出也有符合被預測人的評分清單
rating_in_pred=ratings[ratings['movie_id'].isin(pred_movie_list)]

In [ ]:
#嘗試找出要被預測的電影有沒有不在清單中
print(rating_in_pred[~rating_in_pred['movie_id'].isin(pred_movie_list)])
print(rating_in_pred)

In [ ]:
#找出和預測電影有交集的使用者
match_users=rating_in_pred['user_id'].unique()

In [ ]:
#合併評分和電影集
rating_moive=pd.merge(left=rating_in_pred,right=movies,on='movie_id')
rating_moive=rating_moive.drop("timestamp", axis = 1)

In [ ]:
# Peasoon distance定義
def peasoon_distance(data_xp):
  #對方的平均
  user_avg=data_xp['rating_x'].mean()
  #自己平均
  pred_avg=data_xp['rating_y'].mean()
  corr=0
  coff1=0
  coff2=0
  for data_xx in data_xp.itertuples():
    #對方座標
    prd_x=float(data_xx.rating_x)
    #自己座標
    prd_y=float(data_xx.rating_y)
    corr+=(prd_x-user_avg)*(prd_y-pred_avg)
    coff1+=(prd_x-user_avg)**2
    coff2+=(prd_y-pred_avg)**2
  ps_dist=corr
  if corr!=0:
    ps_dist=corr/(coff1*coff2)**(0.5)
  return ps_dist

In [ ]:
#計算每個使用者的相似度
user_dist=[]
for match_user in match_users:
   data_x=rating_moive[rating_moive['user_id']==match_user]
   data_prd=pd.merge(data_x,pred_user[pred_user['rating']>0], left_on="movie_id", right_on="movie_id")
   user_dist.append([match_user,peasoon_distance(data_prd)])
user_dist_sort=sorted(user_dist, key = lambda x: x[1],reverse=True)

In [ ]:
#找出K個鄰居相似度
k=350
pred_movies=[]

In [ ]:
for user_for_pred in pred_user[pred_user['rating']<0].itertuples():
  pred_movie={}
  pred_movie['movie_id']=user_for_pred.movie_id
  pred_movie['title']=user_for_pred.title
  sim_sum=0
  rate_sum=0
  for i in range(k):
    user=user_dist_sort[i]
    #帳號
    user_id=user[0]
    #相似度
    user_sim=user[1]
    #找到相同的使用者對相同電影評價
    data_user_movies=rating_moive.loc[(rating_moive['user_id']==user_id)&(rating_moive['movie_id']==user_for_pred.movie_id)]
    if not data_user_movies.empty:
      #累加相似度
      sim_sum=sim_sum+user_sim
      #累加評分
      rate_sum=rate_sum+float(data_user_movies['rating'])
  if sim_sum<=0:
    #如果都找不到使用者有啥評價...?
    print(user_for_pred.movie_id)
  #把有相同評價的使用者的評分加總/相似度加總
  raring_pred=round(rate_sum/sim_sum,2)
  pred_movie['rating']=raring_pred
  pred_movies.append(pred_movie)

for  pred in pred_movies:
  print('movie_id: {} , title: {} , rating: {}'.format(pred['movie_id'],pred['title'],pred['rating']))


In [ ]:
while len(pred_user[pred_user['rating']<0])>0:
  user_dis=user_dist_sort.pop(0)
  user=user_dis[0]
  data_user_movies=rating_moive[rating_moive['user_id']==user]
  du_max=data_user_movies['rating'].max()
  du_min=data_user_movies['rating'].min()
  for data_user_movie in data_user_movies.itertuples():
    pred_user.loc[pred_user.movie_id == data_user_movie.movie_id, "rating"]=data_user_movie.rating

In [ ]:
#儲存清洗後的資料
data_clean.to_csv('data_clean_o.csv',index=False)
#儲存清洗後的資料,移除掉header
data_clean.to_csv('data_clean.csv', header=False, index=False)
pred_user.to_csv('pred_user.csv')

In [ ]:
data_clean_o=pd.read_csv('data_clean_o.csv')
pred_user=pd.read_csv('pred_user.csv')

In [ ]:
print(data_clean_o[data_clean_o['user_id']==1])